In [1]:
#importation des packages
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

In [12]:
#chargement du fichier
df = pd.read_csv("Dataset_Final.csv")
df.head()

,CalYear,HourOfCall,DeployedFromLocation,PlusCode_Description,ResponseTime,Day,IncidentGroup,NumCalls,gpe_geo,week,month,DistanceMetrique,PropertyCategory_bis,AddressQualifier_bis
0,2009,0,Home Station,Initial Mobilisation,321.0,Thursday,Special Service,1.0,South West,1,1,1702.197985,Other,Other
1,2009,0,Home Station,Initial Mobilisation,342.0,Thursday,Special Service,1.0,South West,1,1,1351.903485,Other,Other
2,2009,0,Home Station,Initial Mobilisation,308.0,Thursday,Fire,2.0,North West,1,1,624.598820,Other,Other
3,2009,0,Home Station,Initial Mobilisation,210.0,Thursday,Fire,2.0,North West,1,1,332.549762,Other,Other
4,2009,0,Home Station,Initial Mobilisation,233.0,Thursday,Fire,1.0,North East,1,1,1164.431289,Other,Other


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466769 entries, 0 to 2466768
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   CalYear               int64  
 1   HourOfCall            int64  
 2   DeployedFromLocation  object 
 3   PlusCode_Description  object 
 4   ResponseTime          float64
 5   Day                   object 
 6   IncidentGroup         object 
 7   NumCalls              float64
 8   gpe_geo               object 
 9   week                  int64  
 10  month                 int64  
 11  DistanceMetrique      float64
 12  PropertyCategory_bis  object 
 13  AddressQualifier_bis  object 
dtypes: float64(3), int64(4), object(7)
memory usage: 263.5+ MB


In [ ]:
###PREPROCESSING

In [5]:
#récupération d'une partie aléatoire du dataset avec sample uniquement pour la regl log (trop long autrement)
df = df.sample(500000,replace=False , random_state=42)

In [13]:
#remplacement des noms des jours de la variable Day par les chiffres correspondant pour faciliter l'encodage des variables temporelles
df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
df.head(2)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\3932520663.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})


,CalYear,HourOfCall,DeployedFromLocation,PlusCode_Description,ResponseTime,Day,IncidentGroup,NumCalls,gpe_geo,week,month,DistanceMetrique,PropertyCategory_bis,AddressQualifier_bis
0,2009,0,Home Station,Initial Mobilisation,321.0,4,Special Service,1.0,South West,1,1,1702.197985,Other,Other
1,2009,0,Home Station,Initial Mobilisation,342.0,4,Special Service,1.0,South West,1,1,1351.903485,Other,Other


In [14]:
#sélection des variables explicatives et variables cibles
X = df.drop('ResponseTime', axis = 1)
y = df['ResponseTime']

In [15]:
#séparation en jeu d'entrainement et de test, on fixe la séparation avec le paramètre random_state
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
#séparation des colonnes numériques et catégorielles
var_num = ['NumCalls', 'DistanceMetrique']
var_cat = ['CalYear','DeployedFromLocation', 'PlusCode_Description', 'IncidentGroup', 'gpe_geo', 'PropertyCategory_bis', 'AddressQualifier_bis']
var_time = ['HourOfCall', 'week', 'month', 'Day']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_train_time = X_train[var_time]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]
X_test_time = X_test[var_time]

#gestion des variables temps avec sinus et cosinus (variable cyclique)
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

for var in X_train_time :
    X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
    X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
    X_train_time = X_train_time.drop(var, axis=1)

for var in X_test_time :
    X_test_time[var + '_sin'] = sin_transformer(24).fit_transform(X_test_time[var])
    X_test_time[var + '_cos'] = cos_transformer(24).fit_transform(X_test_time[var])
    X_test_time = X_test_time.drop(var, axis=1)

X_train_time.reset_index(drop=True, inplace=True)
X_test_time.reset_index(drop=True, inplace=True)

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat, X_train_time], axis=1)
X_test = pd.concat([X_test_num, X_test_cat, X_test_time], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#affichage de la dimension des jeux reconstitués pour vérifier que l'encodage c'est bien passé
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\3596178187.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\3596178187.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\3596178187.py:27: SettingWithCopyWarning: 
A value is trying to be set on

(1973415, 38)
(493354, 38)
(1973415,)
(493354,)


In [ ]:
###MODELISATION

In [10]:
# Régression Logistique
from sklearn.linear_model import LogisticRegression
reglog = LogisticRegression()
reglog.fit(X_train, y_train)

c:\Users\rivie\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [11]:
joblib.dump(reglog, "model_reglog_cont")

['model_reglog_cont']

In [11]:
#Régression linéaire
from sklearn.linear_model import LinearRegression

reglin = LinearRegression()
reglin.fit(X_train, y_train)

LinearRegression()

In [12]:
joblib.dump(reglin,"model_reglin_cont")

['model_reglin_cont']

In [17]:
#DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

dtreg = DecisionTreeRegressor(random_state=42)
dtreg.fit(X_train, y_train)

DecisionTreeRegressor(random_state=42)

In [22]:
joblib.dump(dtreg, "model_dtreg_cont",compress=9)

['model_dtreg_cont']

In [23]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [24]:
joblib.dump(rf_reg, "model_rfreg_cont",compress=9)

['model_rfreg_cont']

In [17]:
#BayesianRidge
from sklearn import linear_model

ridge = linear_model.BayesianRidge()
ridge.fit(X_train, y_train)

BayesianRidge()

In [18]:
joblib.dump(ridge,"model_ridge_cont")

['model_ridge_cont']

In [19]:
#LassoLars
from sklearn import linear_model

lasso = linear_model.LassoLars(alpha=.1)
lasso.fit(X_train, y_train)

LassoLars(alpha=0.1)

In [20]:
joblib.dump(lasso,"model_lasso_cont")

['model_lasso_cont']

In [ ]:
###Modelisation avec réduction du nb de variable

In [35]:
#chargement du fichier
df_reduc = pd.read_csv('Dataset_Final.csv')

#conservation uniquement de distancemetrique, gpe_geo et de la variable cible
df_reduc = df_reduc[['DistanceMetrique', 'gpe_geo', 'ResponseTime']]

#sélection des variables explicatives et variables cibles
X = df_reduc.drop('ResponseTime', axis = 1)
y = df_reduc['ResponseTime']

#séparation en jeu d'entrainement et de test, on fixe la séparation avec le paramètre random_state
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

##Adaptation du préprocessing qui ne contient pas de variable temporelles
#séparation des colonnes numériques et catégorielles
var_num = ['DistanceMetrique']
var_cat = ['gpe_geo']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat], axis=1)
X_test = pd.concat([X_test_num, X_test_cat], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#affichage de la dimension des jeux reconstitués pour vérifier que l'encodage c'est bien passé
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1973415, 5)
(493354, 5)
(1973415,)
(493354,)


In [36]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor

rf_reg_red = RandomForestRegressor(random_state=42)
rf_reg_red.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [37]:
joblib.dump(rf_reg_red, "model_rfreg_cont_red", compress=9)

['model_rfreg_cont_red']

In [ ]:
###Modelisation avec variable cible entre 200 et 600

In [28]:
common_incidents = pd.read_csv("Dataset_Final.csv")#,sep="\t")
df11 = common_incidents.sample(500000, replace=False)

In [29]:
df12 = df11[(df11["ResponseTime"] >= 200) & (df11["ResponseTime"] <=600)]
df12.dropna(subset=['ResponseTime'], inplace=True)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\1902623342.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12.dropna(subset=['ResponseTime'], inplace=True)


In [30]:
feats = df12.drop('ResponseTime', axis=1)
target = df12['ResponseTime']

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.20, random_state = 42)
# (g)
cat = ["DeployedFromLocation","PlusCode_Description","Day","IncidentGroup","PropertyCategory_bis","AddressQualifier_bis","gpe_geo"]
num = ["NumCalls","DistanceMetrique"]
num_train = X_train[num]
num_test = X_test[num]
cat_train = X_train[cat]
cat_test = X_test[cat]

#variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')

num_train_imputed = imputer_num.fit_transform(num_train)
num_test_imputed = imputer_num.transform(num_test)
#variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_train_imputed = imputer_cat.fit_transform(cat_train)
cat_test_imputed = imputer_cat.transform(cat_test)

# Traitement des variables catégorielles
oneh = OneHotEncoder(drop = 'first', sparse_output=False)
cat_train_imputed_oneh = oneh.fit_transform(cat_train_imputed)
cat_test_imputed_oneh = oneh.transform(cat_test_imputed)
#
num_train = pd.DataFrame(num_train_imputed,columns = num_train.columns)
num_test = pd.DataFrame(num_test_imputed,columns = num_test.columns)
#cat_train = pd.DataFrame(cat_train_imputed_oneh)
# Récupération des noms de colonnes encodées
col_names = oneh.get_feature_names_out(cat)

cat_train = pd.DataFrame(cat_train_imputed_oneh)
cat_test = pd.DataFrame(cat_test_imputed_oneh)
#cat_test = pd.DataFrame(cat_test_imputed_oneh)
X_train = pd.concat([num_train, cat_train], axis = 1)
X_test = pd.concat([num_test, cat_test], axis = 1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

In [41]:
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [42]:
joblib.dump(regressor,"model_regressor_200-600")

['model_regressor_200-600']

In [31]:
RandomForest = RandomForestRegressor(random_state=42)
RandomForest.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [32]:
joblib.dump(RandomForest,"model_RandomForest_200-600", compress=9)

['model_RandomForest_200-600']

In [11]:
from sklearn.linear_model import Ridge
Ridge = Ridge()
Ridge.fit(X_train, y_train)

Ridge()

In [12]:

joblib.dump(Ridge,"model_Ridge_200-600")

['model_Ridge_200-600']

In [13]:
from sklearn.linear_model import Lasso
Lasso = Lasso()
Lasso.fit(X_train, y_train)

Lasso()

In [14]:
joblib.dump(Lasso,"model_Lasso_200-600")

['model_Lasso_200-600']

In [ ]:
###Modelisation avec responsetime >200

In [25]:
common_incidents = pd.read_csv("Dataset_Final.csv")#,sep="\t")
df11 = common_incidents.sample(500000, replace=False)

In [26]:
df12 = df11[df11["ResponseTime"] >= 200]
df12.dropna(subset=['ResponseTime'], inplace=True)

feats = df12.drop('ResponseTime', axis=1)
target = df12['ResponseTime']

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.20, random_state = 42)
# (g)
cat = ["DeployedFromLocation","PlusCode_Description","Day","IncidentGroup","PropertyCategory_bis","AddressQualifier_bis","gpe_geo"]
num = ["NumCalls","DistanceMetrique"]
num_train = X_train[num]
num_test = X_test[num]
cat_train = X_train[cat]
cat_test = X_test[cat]

#variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')

num_train_imputed = imputer_num.fit_transform(num_train)
num_test_imputed = imputer_num.transform(num_test)
#variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_train_imputed = imputer_cat.fit_transform(cat_train)
cat_test_imputed = imputer_cat.transform(cat_test)

# Traitement des variables catégorielles
oneh = OneHotEncoder(drop = 'first', sparse_output=False)
cat_train_imputed_oneh = oneh.fit_transform(cat_train_imputed)
cat_test_imputed_oneh = oneh.transform(cat_test_imputed)
#
num_train = pd.DataFrame(num_train_imputed,columns = num_train.columns)
num_test = pd.DataFrame(num_test_imputed,columns = num_test.columns)
#cat_train = pd.DataFrame(cat_train_imputed_oneh)
# Récupération des noms de colonnes encodées
col_names = oneh.get_feature_names_out(cat)

cat_train = pd.DataFrame(cat_train_imputed_oneh)
cat_test = pd.DataFrame(cat_test_imputed_oneh)
#cat_test = pd.DataFrame(cat_test_imputed_oneh)
X_train = pd.concat([num_train, cat_train], axis = 1)
X_test = pd.concat([num_test, cat_test], axis = 1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\3078214296.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12.dropna(subset=['ResponseTime'], inplace=True)


In [27]:
RandomForest = RandomForestRegressor(random_state=42)
RandomForest.fit(X_train, y_train)
joblib.dump(RandomForest,"model_RandomForest_200", compress=9)

['model_RandomForest_200']

In [19]:
from sklearn.linear_model import Ridge
Ridge = Ridge()
Ridge.fit(X_train, y_train)
joblib.dump(Ridge,"model_Ridge_200")

['model_Ridge_200']

In [21]:
from sklearn.linear_model import Lasso
Lasso = Lasso()
Lasso.fit(X_train, y_train)
joblib.dump(Lasso,"model_Lasso_200")

['model_Lasso_200']

In [ ]:
###Modelisation avec responsetime <600

In [33]:
common_incidents = pd.read_csv("Dataset_Final.csv")#,sep="\t")
df11 = common_incidents.sample(500000, replace=False)

df12 = df11[df11["ResponseTime"] <= 600]
df12.dropna(subset=['ResponseTime'], inplace=True)

feats = df12.drop('ResponseTime', axis=1)
target = df12['ResponseTime']

X_train, X_test, y_train, y_test = train_test_split(feats, target, test_size=0.20, random_state = 42)
# (g)
cat = ["DeployedFromLocation","PlusCode_Description","Day","IncidentGroup","PropertyCategory_bis","AddressQualifier_bis","gpe_geo"]
num = ["NumCalls","DistanceMetrique"]
num_train = X_train[num]
num_test = X_test[num]
cat_train = X_train[cat]
cat_test = X_test[cat]

#variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')

num_train_imputed = imputer_num.fit_transform(num_train)
num_test_imputed = imputer_num.transform(num_test)
#variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
cat_train_imputed = imputer_cat.fit_transform(cat_train)
cat_test_imputed = imputer_cat.transform(cat_test)

# Traitement des variables catégorielles
oneh = OneHotEncoder(drop = 'first', sparse_output=False)
cat_train_imputed_oneh = oneh.fit_transform(cat_train_imputed)
cat_test_imputed_oneh = oneh.transform(cat_test_imputed)
#
num_train = pd.DataFrame(num_train_imputed,columns = num_train.columns)
num_test = pd.DataFrame(num_test_imputed,columns = num_test.columns)
#cat_train = pd.DataFrame(cat_train_imputed_oneh)
# Récupération des noms de colonnes encodées
col_names = oneh.get_feature_names_out(cat)

cat_train = pd.DataFrame(cat_train_imputed_oneh)
cat_test = pd.DataFrame(cat_test_imputed_oneh)
#cat_test = pd.DataFrame(cat_test_imputed_oneh)
X_train = pd.concat([num_train, cat_train], axis = 1)
X_test = pd.concat([num_test, cat_test], axis = 1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\4226906199.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df12.dropna(subset=['ResponseTime'], inplace=True)


In [34]:
RandomForest = RandomForestRegressor(random_state=42)
RandomForest.fit(X_train, y_train)
joblib.dump(RandomForest,"model_RandomForest_600", compress=9)

['model_RandomForest_600']

In [24]:
from sklearn.linear_model import Ridge
Ridge = Ridge()
Ridge.fit(X_train, y_train)
joblib.dump(Ridge,"model_Ridge_600")

['model_Ridge_600']

In [25]:
from sklearn.linear_model import Lasso
Lasso = Lasso()
Lasso.fit(X_train, y_train)
joblib.dump(Lasso,"model_Lasso_600")

['model_Lasso_600']

#Modelisation en catégories

In [ ]:
###Modelisation avec split en 3 catégories

In [38]:
#chargement du fichier final
df = pd.read_csv('Dataset_Final.csv')
#remplacement des noms des jours de la variable Day par les chiffres correspondant
df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
#ajout d'une colonne ResponseTime_cat pour séparer la variable ResponseTime en trois catégories
bin = [0,200,600,1200]
label = ['short', 'medium', 'long']
df['ResponseTime_cat'] = pd.cut(df['ResponseTime'], bins=bin, labels=label)
#suppression de la variable responseTime
df = df.drop('ResponseTime', axis=1)
#sélection des variables explicatives et variables cibles
X = df.drop('ResponseTime_cat', axis = 1)
y = df['ResponseTime_cat']
#séparation en jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#séparation des colonnes numériques et catégorielles
var_num = ['NumCalls', 'DistanceMetrique']
var_cat = ['CalYear','DeployedFromLocation', 'PlusCode_Description', 'IncidentGroup', 'gpe_geo', 'PropertyCategory_bis', 'AddressQualifier_bis']
var_time = ['HourOfCall', 'week', 'month', 'Day']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_train_time = X_train[var_time]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]
X_test_time = X_test[var_time]

#gestion des variables temps avec sinus et cosinus (variable cyclique)
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

for var in X_train_time :
    X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
    X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
    X_train_time = X_train_time.drop(var, axis=1)

for var in X_test_time :
    X_test_time[var + '_sin'] = sin_transformer(24).fit_transform(X_test_time[var])
    X_test_time[var + '_cos'] = cos_transformer(24).fit_transform(X_test_time[var])
    X_test_time = X_test_time.drop(var, axis=1)

X_train_time.reset_index(drop=True, inplace=True)
X_test_time.reset_index(drop=True, inplace=True)

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat, X_train_time], axis=1)
X_test = pd.concat([X_test_num, X_test_cat, X_test_time], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#encodage de la variable cible ResponseTime_cat dans l'ordre short, medium, long
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#affichage de la dimension des jeux reconstitués
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\624767693.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\624767693.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_6412\624767693.py:38: SettingWithCopyWarning

(1973415, 38)
(493354, 38)
(1973415,)
(493354,)


In [28]:
#Régression logistic
from sklearn.linear_model import LogisticRegression
reglog_cat = LogisticRegression()
reglog_cat.fit(X_train, y_train)
joblib.dump(reglog_cat,"model_reglog_3cat")

['model_reglog_3cat']

In [29]:
#DecisionTreeClassifier
from sklearn.tree import DecisionTreeClassifier
dtc_3cat = DecisionTreeClassifier(max_depth = 3, random_state=42)
dtc_3cat.fit(X_train, y_train)
joblib.dump(dtc_3cat, "model_dtc_3cat")

['model_dtc_3cat']

In [30]:
#RidgeClassifier
from sklearn import linear_model
rc_3cat = linear_model.RidgeClassifier()
rc_3cat.fit(X_train, y_train)
joblib.dump(rc_3cat, "model_rc_3cat")

['model_rc_3cat']

In [39]:
#RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
rfc_3cat = RandomForestClassifier()
rfc_3cat.fit(X_train, y_train)
joblib.dump(rfc_3cat, "model_rfc_3cat", compress=9)

['model_rfc_3cat']

In [36]:
#MLPClassifier
from sklearn.neural_network import MLPClassifier
mlpc_3cat = MLPClassifier()
mlpc_3cat.fit(X_train, y_train)
joblib.dump(mlpc_3cat, "model_mlpc_3cat")

['model_mlpc_3cat']

In [37]:
#GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier()
gbc.fit(X_train, y_train)

GradientBoostingClassifier()

In [38]:
joblib.dump(gbc, "model_gbc_3cat")

['model_gbc_3cat']

In [ ]:
###Modelisation avec split en 7 catégories

In [39]:
#chargement du fichier final
df = pd.read_csv('Dataset_Final.csv')
#remplacement des noms des jours de la variable Day par les chiffres correspondant
df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
#ajout d'une colonne ResponseTime_cat pour séparer la variable ResponseTime en 7 catégories
bin = [0,75,150,450,600,800,900,1200]
label = ['xveryshort','veryshort','short', 'medium','xmedium', 'long','xlong']
df['ResponseTime_cat'] = pd.cut(df['ResponseTime'], bins=bin, labels=label)
#suppression de la variable responseTime
df = df.drop('ResponseTime', axis=1)
#sélection des variables explicatives et variables cibles
X = df.drop('ResponseTime_cat', axis = 1)
y = df['ResponseTime_cat']
#séparation en jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#séparation des colonnes numériques et catégorielles
var_num = ['NumCalls', 'DistanceMetrique']
var_cat = ['CalYear','DeployedFromLocation', 'PlusCode_Description', 'IncidentGroup', 'gpe_geo', 'PropertyCategory_bis', 'AddressQualifier_bis']
var_time = ['HourOfCall', 'week', 'month', 'Day']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_train_time = X_train[var_time]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]
X_test_time = X_test[var_time]

#gestion des variables temps avec sinus et cosinus (variable cyclique)
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

for var in X_train_time :
    X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
    X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
    X_train_time = X_train_time.drop(var, axis=1)

for var in X_test_time :
    X_test_time[var + '_sin'] = sin_transformer(24).fit_transform(X_test_time[var])
    X_test_time[var + '_cos'] = cos_transformer(24).fit_transform(X_test_time[var])
    X_test_time = X_test_time.drop(var, axis=1)

X_train_time.reset_index(drop=True, inplace=True)
X_test_time.reset_index(drop=True, inplace=True)

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat, X_train_time], axis=1)
X_test = pd.concat([X_test_num, X_test_cat, X_test_time], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#encodage de la variable cible ResponseTime_cat dans l'ordre short, medium, long
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#affichage de la dimension des jeux reconstitués
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\3740983920.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\3740983920.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\3740983920.py:38: SettingWithCopyW

(1973415, 38)
(493354, 38)
(1973415,)
(493354,)


In [40]:
#Régression logistic
from sklearn.linear_model import LogisticRegression
reglog_7cat = LogisticRegression()
reglog_7cat.fit(X_train, y_train)
joblib.dump(reglog_7cat, "model_reglog_7cat")

c:\Users\rivie\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


['model_reglog_7cat']

In [ ]:
###Modelisation avec split en 5 catégories

In [41]:
#chargement du fichier final
df = pd.read_csv('Dataset_Final.csv')
#remplacement des noms des jours de la variable Day par les chiffres correspondant
df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
#ajout d'une colonne ResponseTime_cat pour séparer la variable ResponseTime en 5 catégories
bin = [0,150,450,600,800,1200]
label = ['veryshort','short', 'medium','xmedium', 'long']
df['ResponseTime_cat'] = pd.cut(df['ResponseTime'], bins=bin, labels=label)
#suppression de la variable responseTime
df = df.drop('ResponseTime', axis=1)
#sélection des variables explicatives et variables cibles
X = df.drop('ResponseTime_cat', axis = 1)
y = df['ResponseTime_cat']
#séparation en jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#séparation des colonnes numériques et catégorielles
var_num = ['NumCalls', 'DistanceMetrique']
var_cat = ['CalYear','DeployedFromLocation', 'PlusCode_Description', 'IncidentGroup', 'gpe_geo', 'PropertyCategory_bis', 'AddressQualifier_bis']
var_time = ['HourOfCall', 'week', 'month', 'Day']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_train_time = X_train[var_time]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]
X_test_time = X_test[var_time]

#gestion des variables temps avec sinus et cosinus (variable cyclique)
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

for var in X_train_time :
    X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
    X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
    X_train_time = X_train_time.drop(var, axis=1)

for var in X_test_time :
    X_test_time[var + '_sin'] = sin_transformer(24).fit_transform(X_test_time[var])
    X_test_time[var + '_cos'] = cos_transformer(24).fit_transform(X_test_time[var])
    X_test_time = X_test_time.drop(var, axis=1)

X_train_time.reset_index(drop=True, inplace=True)
X_test_time.reset_index(drop=True, inplace=True)

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat, X_train_time], axis=1)
X_test = pd.concat([X_test_num, X_test_cat, X_test_time], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#encodage de la variable cible ResponseTime_cat dans l'ordre short, medium, long
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#affichage de la dimension des jeux reconstitués
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\852829459.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\852829459.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\852829459.py:38: SettingWithCopyWarn

(1973415, 38)
(493354, 38)
(1973415,)
(493354,)


In [42]:
#Régression logistic
from sklearn.linear_model import LogisticRegression
reglog_5cat = LogisticRegression()
reglog_5cat.fit(X_train, y_train)
joblib.dump(reglog_5cat, "model_reglog_5cat")

['model_reglog_5cat']

In [ ]:
###Modelisation avec split en 2 catégories

In [43]:
#chargement du fichier final
df = pd.read_csv('Dataset_Final.csv')
#remplacement des noms des jours de la variable Day par les chiffres correspondant
df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
#ajout d'une colonne ResponseTime_cat pour séparer la variable ResponseTime en 7 catégories
bin = [0,450,1200]
label = ['short', 'medium-long']
df['ResponseTime_cat'] = pd.cut(df['ResponseTime'], bins=bin, labels=label)
#suppression de la variable responseTime
df = df.drop('ResponseTime', axis=1)
#sélection des variables explicatives et variables cibles
X = df.drop('ResponseTime_cat', axis = 1)
y = df['ResponseTime_cat']
#séparation en jeu d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#séparation des colonnes numériques et catégorielles
var_num = ['NumCalls', 'DistanceMetrique']
var_cat = ['CalYear','DeployedFromLocation', 'PlusCode_Description', 'IncidentGroup', 'gpe_geo', 'PropertyCategory_bis', 'AddressQualifier_bis']
var_time = ['HourOfCall', 'week', 'month', 'Day']

X_train_num = X_train[var_num]
X_train_cat = X_train[var_cat]
X_train_time = X_train[var_time]
X_test_num = X_test[var_num]
X_test_cat = X_test[var_cat]
X_test_time = X_test[var_time]

#gestion des variables temps avec sinus et cosinus (variable cyclique)
from sklearn.preprocessing import FunctionTransformer
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

for var in X_train_time :
    X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
    X_train_time[var + '_cos'] = cos_transformer(24).fit_transform(X_train_time[var])
    X_train_time = X_train_time.drop(var, axis=1)

for var in X_test_time :
    X_test_time[var + '_sin'] = sin_transformer(24).fit_transform(X_test_time[var])
    X_test_time[var + '_cos'] = cos_transformer(24).fit_transform(X_test_time[var])
    X_test_time = X_test_time.drop(var, axis=1)

X_train_time.reset_index(drop=True, inplace=True)
X_test_time.reset_index(drop=True, inplace=True)

#remplissage des valeurs manquantes par simple imputer avec la stratégie median pour les variables numériques et le most frequent pour les variables catégorielles
import numpy as np
from sklearn.impute import SimpleImputer

#gestion des données manquantes pour les variables numériques
imputer_num = SimpleImputer(missing_values=np.nan, strategy='median')
X_train_num = imputer_num.fit_transform(X_train_num)
X_test_num = imputer_num.transform(X_test_num)

#gestion des données manquantes pour les variables catégorielles
imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
X_train_cat = imputer_cat.fit_transform(X_train_cat)
X_test_cat = imputer_cat.transform(X_test_cat)

#standardisation des variables numériques avec StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_num = scaler.fit_transform(X_train_num)
X_test_num = scaler.transform(X_test_num)

#encodage des variables catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(drop = 'first', sparse_output=False)
X_train_cat = encoder.fit_transform(X_train_cat)
X_test_cat = encoder.transform(X_test_cat)

##passage en dataframe des tableaux récupérés après encodage
X_train_num = pd.DataFrame(X_train_num)
X_test_num = pd.DataFrame(X_test_num)
X_train_cat = pd.DataFrame(X_train_cat)
X_test_cat = pd.DataFrame(X_test_cat)

#concaténation des jeux d'entraînement et de test
X_train = pd.concat([X_train_num, X_train_cat, X_train_time], axis=1)
X_test = pd.concat([X_test_num, X_test_cat, X_test_time], axis=1)

X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)

#encodage de la variable cible ResponseTime_cat dans l'ordre short, medium, long
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

#affichage de la dimension des jeux reconstitués
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\2692739086.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['Day'] = df['Day'].replace({'Monday': 1, 'Tuesday': 2, 'Wednesday': 3, 'Thursday': 4, 'Friday': 5, 'Saturday': 6, 'Sunday': 7})
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\2692739086.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train_time[var + '_sin'] = sin_transformer(24).fit_transform(X_train_time[var])
C:\Users\rivie\AppData\Local\Temp\ipykernel_20448\2692739086.py:38: SettingWithCopyW

(1973415, 38)
(493354, 38)
(1973415,)
(493354,)


In [44]:
#Régression logistic
from sklearn.linear_model import LogisticRegression
reglog_2cat = LogisticRegression()
reglog_2cat.fit(X_train, y_train)
joblib.dump(reglog_2cat, "model_reglog_2cat")

['model_reglog_2cat']